In [ ]:
# @title
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import pytz

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def findTimes(url):
    driver = web_driver()

    driver.get(url)
    url_source = driver.page_source
    driver.quit()

    html = BeautifulSoup(url_source, 'html.parser')
    times = []
    titles = []
    for block in html.find_all('div', class_='todays-hrs-block'):
        title = block.find('p', class_='hourstitle').text
        start_time = block.find('div', class_='start-hours').text.strip()
        end_time = block.find('div', class_='end-hours').text.strip()
        titles.append(len(title))
        times.append([title, start_time[:-2], end_time])

    printStringTimes = ""

    maxTitle = max(titles)
    space = maxTitle+5

    for i in times:
        printStringTimes += "{}:{} {} ~ {}\n".format(i[0], "-"*(space-len(i[0])), i[1], i[2])
    return [printStringTimes, times]


def checkTime(curr_h,curr_m,time_start_h,time_end_h,time_start_m,time_end_m, meal_data):
    if curr_h > time_end_h or curr_h < time_start_h:
        return "continue"
    elif curr_h == time_end_h or curr_h == time_start_h:
        if curr_h == time_end_h and curr_m >= time_end_m:
            return "continue"
        elif curr_h == time_start_h and curr_m < time_start_m:
            return "continue"
        elif curr_h == time_end_h and curr_m < time_end_m:
            #good
            return [True,meal_data]
        elif curr_h == time_start_h and curr_m >= time_start_m:
            #good
            return [True,meal_data]
    elif curr_h < time_end_h and curr_h >= time_start_h:
        return [True,meal_data]

def isOpen(data, current_time):
    for x in data:
        meal_data = []
        meal_data.append(x[0])
        t1 = x[1]
        t2 = x[2]
        if t1[-2] == "P":
            time_start_h = int(t1[:-2].split(":")[0])+12
        else:
            time_start_h = int(t1[:-2].split(":")[0])


        if t2[-2] == "P":
            time_end_h = int(t2[:-2].split(":")[0])+12
        else:
            time_end_h = int(t2[:-2].split(":")[0])

        time_start_m = int(t1[:-2].split(":")[1])
        time_end_m = int(t2[:-2].split(":")[1])

        meal_data.append(time_start_h)
        meal_data.append(time_start_m)
        meal_data.append(time_end_h)
        meal_data.append(time_end_m)


        curr_h = int(current_time.split(":")[0])
        curr_m = int(current_time.split(":")[1])
        cont = checkTime(curr_h,curr_m,time_start_h,time_end_h,time_start_m,time_end_m, meal_data)
        if cont == "continue":
            continue
        else:
            return cont

    return [False,data]


def showOutput(currTime, locationName, url):
    data = findTimes(url)
    checkOpen = isOpen(data[1], currTime)
    print_string = "\t{} Hours:\n".format(locationName)+data[0]


    if checkOpen[0]:
        h = checkOpen[1][-2]
        m = checkOpen[1][-1]
        ch = int(currTime.split(":")[0])
        cm = int(currTime.split(":")[1])

        difftot = (h*60 + m) - (ch*60 + cm)
        diffh = difftot//60
        diffm = difftot%60

        suf = "AM"
        if h > 12:
            h-=12
            suf = "PM"
        if m<10:
            m = "0"+str(m)


        ret = "{} is open at {} until {}:{}{}.".format(checkOpen[1][0], locationName, h,m,suf)
        if diffh>0:
            return [print_string, ret + " {} hour(s) and {} minute(s) away\n".format(diffh, diffm)]
        else:
            return [print_string, ret + " {} minute(s) away\n".format(diffm)]
    else:
        store = {}
        name_start = {}
        ch = int(currTime.split(":")[0])
        cm = int(currTime.split(":")[1])
        for t in checkOpen[1]:
            h = int(t[-2].split(":")[0])
            if t[-2].split(":")[1][-2:] == "PM":
                h+=12
            m = int(t[-2].split(":")[1][:-2])

            difftot = (h*60 + m) - (ch*60 + cm)
            store[t[0]] = difftot
            name_start[difftot] = t[-2]
        nearest_time = min(filter(lambda x: x > 0, store.values()), default=min(store.values()))
        name = list(store.keys())[list(store.values()).index(nearest_time)]
        h = nearest_time//60
        m = nearest_time%60
        if h>0:
            return [print_string, "{} is not open. {} will be open at {}. {} hour(s) and {} minute(s)\n".format(locationName, name, name_start.get(nearest_time), h, m)]
        elif h<0:
            morningTime = int(name_start.get(nearest_time).split(":")[0])*60+int(name_start.get(nearest_time).split(":")[1][:-2])
            t = 24*60-ch*60-cm+ morningTime
            h = t//60
            m = t%60
            return [print_string, "{} is not open. {} will be open at {}. {} hour(s) and {} minute(s)\n".format(locationName, name, name_start.get(nearest_time), h, m)]
        else:
            return [print_string, "{} is not open. {} will be open at {}. {} minute(s)\n".format(locationName, name, name_start.get(nearest_time), m)]



now = datetime.now(pytz.timezone("America/New_York"))
current_time = now.strftime("%H:%M")
print(current_time)

sage = showOutput(current_time, "Sage", "https://rpi.sodexomyway.com/dining-near-me/russell-sage")
commons = showOutput(current_time, "Commons", "https://rpi.sodexomyway.com/dining-near-me/commons-dining-hall")

print("\n")
print(sage[0])
print(commons[0])
print()
print(sage[1])
print(commons[1])

def ret():
    return ("{}\n{}\n{}\n{}".format(sage[0],commons[0],sage[1],commons[1]),current_time)


15:55


	Sage Hours:
Breakfast:----------------- 7:00AM ~ 10:00AM
Continental Breakfast:----- 10:00AM ~ 11:00AM
Lunch:--------------------- 11:00AM ~ 2:30PM
Dinner:-------------------- 4:00PM ~ 8:00PM

	Commons Hours:
Breakfast:----------------- 7:00AM ~ 10:00AM
Continental Breakfast:----- 10:00AM ~ 11:00AM
Lunch:--------------------- 11:00AM ~ 3:30PM
Dinner:-------------------- 4:30PM ~ 9:00PM


Sage is not open. Dinner will be open at 4:00PM. 5 minute(s)

Commons is not open. Dinner will be open at 4:30PM. 35 minute(s)

